In [18]:
from bs4 import BeautifulSoup
import requests
import time

# Define the function which will scrape data about a person and return it 
def get_person_data(url):
    an_actor = BeautifulSoup(requests.get(url).text, 'html.parser')
    info = an_actor.find('table', attrs={'id': 'name-overview-widget-layout'})
    res = {}
    image = info.find('div', 'image').find('img')
    if image is not None:
        res['image_url'] = image['src']
    else: 
        res['image_url'] = ''
    birth_data = info.find('div', attrs={'id': 'name-born-info'})
    if birth_data is not None:
        res['born'] = birth_data.find('time')['datetime']
        birth_place = birth_data.findAll('a')[-1].contents[0]
        birth_place = birth_data.findAll('a')[-1].contents[0].split(',')
        res['country'] = birth_place[-1]
        res['city'] = birth_place[0]
    else:
        res['born'] = res['country'] = res['city'] = ''    
    try:
        death_data = info.find('div', attrs={'id': 'name-death-info'})
        res['died'] = death_data.find('time')['datetime']
    except:
        res['died'] = ''
    return res


def crawl_page(startswith):
    url = "http://www.imdb.com/search/title?sort=num_votes,desc&start={}&title_type=feature&year=1900,2015".format(startswith)
    r = requests.get(url)
    data = []
    bs = BeautifulSoup(r.text, 'html.parser')
    for num, movie in enumerate(bs.findAll('td','title')):
        if (num + 1) % 10 == 0 and num > 0:
            print num + 1
            
        genres = movie.find('span','genre').findAll('a')
        
        dirs, acts = str(movie.find('span','credit')).split("With:")
        dirs = BeautifulSoup(dirs + '</span>')
        acts = BeautifulSoup('<span>' + acts)

        directors = []
        for i in dirs.findAll('a'):
            directors.append(get_person_data('http://www.imdb.com' + i['href']))

        actors = []
        for i in acts.findAll('a'):
            actors.append(get_person_data('http://www.imdb.com' + i['href']))

        # Let's collect each movie data into a dictionary
        data.append({
            'title': movie.find('a').contents[0],
            'genres': [g.contents[0] for g in genres],
            'runtime': movie.find('span','runtime').contents[0].split()[0],
            'rating': movie.find('span','value').contents[0],
            'released': movie.find('span','year_type').contents[0][1:-1],
            'description':  movie.find('span', 'outline').contents[0],
            'directors': directors,
            'actors': actors,
        })
    return data

def collect_data(start_page, end_page):
    data = []
    start = time.time()
    for i, j in enumerate(range(start_page, end_page, 50)):
        diff = time.time() - start
        print '\nPage', i+1, "{} min {} sec".format(int(diff // 60), int(diff % 60))
        data.extend(crawl_page(i))
    return data

#print "\nData about {} movies were collected".format(len(data))

In [ ]:
data = collect_data(1, 1501)

In [41]:
import json
with open('imdb_movies_1500.json', 'w') as f:
    json.dump(data, f)